In [2]:
#import rdflib
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import DC, RDF, FOAF, RDFS, XSD #import already in RDFlib integrated namespaces 
#for visualization
import io
import pydotplus
from IPython.display import display, Image
from rdflib.tools.rdf2dot import rdf2dot
from rdflib import Graph, Namespace, RDF, RDFS, URIRef
import pandas as pd #for handling csv and csv contents
from rdflib import Graph, Literal, RDF, URIRef, Namespace #basic RDF handling
from rdflib.namespace import FOAF , XSD #most common namespaces
import urllib.parse #for parsing strings to URI's

In [3]:
url='data/author.csv'
df=pd.read_csv(url,sep="|")
df.head()

,authorId:ID,authorName:string
0,2144794493,Jie Huang
1,2302556600,Cheng-Xiang Wang
2,2075396941,L. Bai
3,2152916916,Yang Yang
4,2149869431,Jie Li


In [ ]:
g = Graph()
DBO = Namespace("http://dbpedia.org/ontology/")

In [ ]:
for index, row in df.iterrows():
    print(row['authorName:string'])
    #g.add((URIRef(DBO+row['authorName']), RDF.type, FOAF.Person))
    #g.add((URIRef(DBO+row['Name']), URIRef(schema+'name'), Literal(row['Name'], datatype=XSD.string) ))

KeyError: 'authorName'